In [1]:
# Install Dependencies
from nba_api.stats.endpoints import playercareerstats # Provides single player career stats"
from nba_api.live.nba.endpoints import scoreboard # Provides scoreboards for that day"
from nba_api.stats.static import players # Receive player information i.e. player_id
from nba_api.stats.static import teams # Retrieve team data
from nba_api.stats.endpoints import leaguegamefinder # Retrieve specific games
from nba_api.stats.endpoints import leagueleaders # Retrieve stat leaders for a season
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import shotchartdetail
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Gather The Data

In [2]:
# Get the data for all nba teams
# Get all players for Coach Pops first year as a head coach 1996-97
players_9697 = leagueleaders.LeagueLeaders(
    season='1996-97',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:600]
# Show the top 50 players
players_9697.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,893,1,Michael Jordan,1610612741,CHI,82,3109,920,1892,0.486,...,482,352,140,44,166,156,2431,2215,2.12,0.84
1,252,2,Karl Malone,1610612762,UTA,82,2998,864,1571,0.550,...,809,368,113,48,233,217,2249,2478,1.58,0.48
2,779,3,Glen Rice,1610612766,CHH,79,3359,722,1513,0.477,...,318,160,72,26,177,190,2115,1652,0.90,0.41
3,782,4,Mitch Richmond,1610612758,SAC,81,3115,717,1578,0.454,...,319,338,118,24,237,211,2095,1722,1.43,0.50
4,84,5,Latrell Sprewell,1610612744,GSW,80,3350,649,1444,0.449,...,366,507,132,45,322,153,1938,1779,1.57,0.41


In [3]:
spurs_9697 = players_9697.loc[players_9697['TEAM']=='SAS']
spurs_9697.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
61,1122,62,Dominique Wilkins,1610612759,SAS,63,1947,397,953,0.417,...,402,119,39,31,135,100,1145,976,0.88,0.29
91,137,92,Vernon Maxwell,1610612759,SAS,72,2077,340,906,0.375,...,159,153,87,19,121,168,929,614,1.26,0.72
101,219,102,Vinny Del Negro,1610612759,SAS,72,2238,365,781,0.467,...,210,231,59,7,92,131,886,868,2.51,0.64
118,422,119,Avery Johnson,1610612759,SAS,76,2473,327,685,0.477,...,147,513,96,15,146,158,800,1004,3.51,0.66
151,61,151,Carl Herrera,1610612759,SAS,75,1831,257,593,0.433,...,340,50,62,53,95,217,597,634,0.53,0.65


In [4]:
# insert a year category
spurs_9697.insert(6, 'Year', '1996-97')

spurs_9697

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,Year,MIN,FGM,FGA,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
61,1122,62,Dominique Wilkins,1610612759,SAS,63,1996-97,1947,397,953,...,402,119,39,31,135,100,1145,976,0.88,0.29
91,137,92,Vernon Maxwell,1610612759,SAS,72,1996-97,2077,340,906,...,159,153,87,19,121,168,929,614,1.26,0.72
101,219,102,Vinny Del Negro,1610612759,SAS,72,1996-97,2238,365,781,...,210,231,59,7,92,131,886,868,2.51,0.64
118,422,119,Avery Johnson,1610612759,SAS,76,1996-97,2473,327,685,...,147,513,96,15,146,158,800,1004,3.51,0.66
151,61,151,Carl Herrera,1610612759,SAS,75,1996-97,1831,257,593,...,340,50,62,53,95,217,597,634,0.53,0.65
152,42,153,Monty Williams,1610612759,SAS,65,1996-97,1347,234,460,...,206,91,55,52,116,161,588,584,0.78,0.47
154,251,154,Sean Elliott,1610612759,SAS,39,1996-97,1393,196,464,...,190,124,24,24,89,105,582,539,1.39,0.27
155,724,156,Cory Alexander,1610612759,SAS,80,1996-97,1451,194,490,...,123,254,82,16,146,148,577,576,1.74,0.56
160,781,161,Will Perdue,1610612759,SAS,65,1996-97,1919,233,410,...,638,38,32,102,87,184,565,1039,0.44,0.37
234,246,236,Greg Anderson,1610612759,SAS,82,1996-97,1662,130,262,...,448,34,63,67,73,225,322,698,0.47,0.86


In [5]:
# Get the data for all nba teams
# Get all players for Coach Pops first championship as a head coach 1998-99
players_9899 = leagueleaders.LeagueLeaders(
    season='1998-99',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:600]
# Show the top 50 players
players_9899.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,406,1,Shaquille O'Neal,1610612747,LAL,49,1705,510,885,0.576,...,525,114,36,82,122,155,1289,1320,0.93,0.29
1,947,2,Allen Iverson,1610612755,PHI,48,1990,435,1056,0.412,...,236,223,110,7,167,98,1284,954,1.33,0.66
2,252,3,Karl Malone,1610612762,UTA,49,1832,393,797,0.493,...,463,201,62,28,162,134,1164,1250,1.24,0.38
3,949,4,Shareef Abdur-Rahim,1610612763,VAN,50,2021,386,893,0.432,...,374,172,69,55,186,137,1152,1059,0.93,0.37
4,56,5,Gary Payton,1610612760,SEA,50,2008,401,923,0.434,...,244,436,109,12,154,115,1084,1132,2.83,0.71


In [6]:
# Get spurs team for 98-99
spurs_9899 = players_9899.loc[players_9899['TEAM']=='SAS']

spurs_9899.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
5,1495,5,Tim Duncan,1610612759,SAS,50,1963,418,845,0.495,...,571,121,45,126,146,147,1084,1263,0.83,0.31
32,764,33,David Robinson,1610612759,SAS,49,1554,268,527,0.509,...,492,103,69,119,108,143,775,1067,0.95,0.64
75,251,75,Sean Elliott,1610612759,SAS,50,1509,208,507,0.410,...,213,117,26,17,71,104,561,530,1.65,0.37
99,422,100,Avery Johnson,1610612759,SAS,50,1672,218,461,0.473,...,118,369,51,11,112,101,487,643,3.29,0.46
117,53,116,Mario Elie,1610612759,SAS,47,1291,156,331,0.471,...,137,89,46,12,61,91,455,487,1.46,0.75


In [7]:
# Insert year column
spurs_9899.insert(6, 'Year', '1998-99')

spurs_9899

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,Year,MIN,FGM,FGA,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
5,1495,5,Tim Duncan,1610612759,SAS,50,1998-99,1963,418,845,...,571,121,45,126,146,147,1084,1263,0.83,0.31
32,764,33,David Robinson,1610612759,SAS,49,1998-99,1554,268,527,...,492,103,69,119,108,143,775,1067,0.95,0.64
75,251,75,Sean Elliott,1610612759,SAS,50,1998-99,1509,208,507,...,213,117,26,17,71,104,561,530,1.65,0.37
99,422,100,Avery Johnson,1610612759,SAS,50,1998-99,1672,218,461,...,118,369,51,11,112,101,487,643,3.29,0.46
117,53,116,Mario Elie,1610612759,SAS,47,1998-99,1291,156,331,...,137,89,46,12,61,91,455,487,1.46,0.75
186,1114,187,Jaren Jackson,1610612759,SAS,47,1998-99,861,108,284,...,99,49,41,9,37,63,301,279,1.32,1.11
191,990,192,Malik Rose,1610612759,SAS,47,1998-99,608,93,201,...,182,29,40,22,56,120,284,345,0.52,0.71
226,1498,227,Antonio Daniels,1610612759,SAS,47,1998-99,614,83,183,...,54,106,30,6,44,39,220,256,2.41,0.68
244,70,245,Steve Kerr,1610612759,SAS,44,1998-99,734,68,174,...,44,49,23,3,22,28,192,179,2.23,1.04
270,760,271,Jerome Kersey,1610612759,SAS,45,1998-99,699,68,200,...,130,41,37,14,30,92,145,197,1.37,1.23


In [8]:
# Get the data for all nba teams
# Get all players for Coach Pops second chip as a head coach 2002-03
players_0203 = leagueleaders.LeagueLeaders(
    season='2002-03',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:600]
# Show the top 50 players
players_0203.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,977,1,Kobe Bryant,1610612747,LAL,82,3401,868,1924,0.451,...,564,481,181,67,288,218,2461,2298,1.67,0.63
1,1503,2,Tracy McGrady,1610612753,ORL,75,2954,829,1813,0.457,...,488,411,124,59,195,156,2407,2160,2.11,0.64
2,947,3,Allen Iverson,1610612755,PHI,82,3485,804,1940,0.414,...,344,454,225,13,286,149,2262,1710,1.59,0.79
3,1718,4,Paul Pierce,1610612738,BOS,79,3096,663,1592,0.416,...,578,349,139,62,288,227,2048,1810,1.21,0.48
4,1717,5,Dirk Nowitzki,1610612742,DAL,80,3117,690,1489,0.463,...,791,239,111,82,152,206,2011,2218,1.57,0.73


In [9]:
# Get the spurs team for the 02-03 season
spurs_0203 = players_0203.loc[players_0203['TEAM']=='SAS']

spurs_0203.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
5,1495,6,Tim Duncan,1610612759,SAS,81,3181,714,1392,0.513,...,1043,316,55,237,248,231,1884,2425,1.27,0.22
43,2225,44,Tony Parker,1610612759,SAS,82,2774,484,1043,0.464,...,216,432,71,4,198,174,1269,1164,2.18,0.36
73,1536,74,Stephen Jackson,1610612759,SAS,80,2254,356,818,0.435,...,286,183,125,30,176,202,946,888,1.04,0.71
100,990,101,Malik Rose,1610612759,SAS,79,1933,289,630,0.459,...,506,124,57,40,170,206,822,974,0.73,0.34
154,1477,155,Bruce Bowen,1610612759,SAS,82,2566,223,479,0.466,...,239,113,66,42,72,195,583,662,1.57,0.92


In [10]:
# Insert the year so that it will be able to be grouped by later
spurs_0203.insert(6, 'Year', '2002-03')
spurs_0203

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,Year,MIN,FGM,FGA,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
5,1495,6,Tim Duncan,1610612759,SAS,81,2002-03,3181,714,1392,...,1043,316,55,237,248,231,1884,2425,1.27,0.22
43,2225,44,Tony Parker,1610612759,SAS,82,2002-03,2774,484,1043,...,216,432,71,4,198,174,1269,1164,2.18,0.36
73,1536,74,Stephen Jackson,1610612759,SAS,80,2002-03,2254,356,818,...,286,183,125,30,176,202,946,888,1.04,0.71
100,990,101,Malik Rose,1610612759,SAS,79,2002-03,1933,289,630,...,506,124,57,40,170,206,822,974,0.73,0.34
154,1477,155,Bruce Bowen,1610612759,SAS,82,2002-03,2566,223,479,...,239,113,66,42,72,195,583,662,1.57,0.92
164,764,165,David Robinson,1610612759,SAS,64,2002-03,1676,197,420,...,508,61,52,111,83,126,546,910,0.73,0.63
171,1938,172,Manu Ginobili,1610612759,SAS,69,2002-03,1431,174,397,...,161,138,96,17,100,170,525,569,1.38,0.96
228,120,229,Steven Smith,1610612759,SAS,53,2002-03,1032,113,291,...,99,70,28,9,43,79,360,326,1.63,0.65
246,70,247,Steve Kerr,1610612759,SAS,75,2002-03,952,110,256,...,60,70,27,3,35,49,299,274,2.00,0.77
248,788,249,Kevin Willis,1610612759,SAS,71,2002-03,840,123,257,...,226,24,20,20,60,120,297,361,0.40,0.33


In [11]:
# Get the data for all nba teams
# Get all players for Coach Pops third championship as a head coach 2004-05
players_0405 = leagueleaders.LeagueLeaders(
    season='2004-05',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:600]
# Show the top 50 players
players_0405.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,947,1,Allen Iverson,1610612755,PHI,75,3173,771,1818,0.424,...,299,596,180,9,344,140,2302,1865,1.73,0.52
1,2544,2,LeBron James,1610612739,CLE,80,3388,795,1684,0.472,...,588,577,177,52,262,146,2175,2259,2.20,0.68
2,2405,3,Amar'e Stoudemire,1610612756,PHX,80,2886,747,1336,0.559,...,713,131,77,130,189,278,2080,2141,0.69,0.41
3,2240,4,Gilbert Arenas,1610612764,WAS,80,3274,656,1523,0.431,...,378,411,139,23,242,245,2038,1761,1.70,0.57
4,1717,5,Dirk Nowitzki,1610612742,DAL,78,3020,663,1445,0.459,...,757,240,97,119,176,219,2032,2194,1.36,0.55


In [12]:
# Get the spurs team for the 04-05 season
spurs_0405 = players_0405.loc[players_0405['TEAM']=='SAS']

spurs_0405.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
31,1495,32,Tim Duncan,1610612759,SAS,66,2203,517,1042,0.496,...,732,179,45,174,127,144,1342,1670,1.41,0.35
33,2225,34,Tony Parker,1610612759,SAS,80,2735,539,1118,0.482,...,298,491,98,4,215,167,1331,1315,2.28,0.46
48,1938,49,Manu Ginobili,1610612759,SAS,74,2193,367,780,0.471,...,329,288,119,27,172,190,1186,1277,1.67,0.69
128,1737,129,Nazr Mohammed,1610612759,SAS,77,1935,289,602,0.480,...,586,34,58,86,108,236,731,1000,0.32,0.54
144,1477,145,Bruce Bowen,1610612759,SAS,82,2628,251,598,0.420,...,285,126,55,39,57,175,675,735,2.21,0.96


In [13]:
# Insert year
spurs_0405.insert(6, 'Year', '2004-05')

spurs_0405

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,Year,MIN,FGM,FGA,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
31,1495,32,Tim Duncan,1610612759,SAS,66,2004-05,2203,517,1042,...,732,179,45,174,127,144,1342,1670,1.41,0.35
33,2225,34,Tony Parker,1610612759,SAS,80,2004-05,2735,539,1118,...,298,491,98,4,215,167,1331,1315,2.28,0.46
48,1938,49,Manu Ginobili,1610612759,SAS,74,2004-05,2193,367,780,...,329,288,119,27,172,190,1186,1277,1.67,0.69
128,1737,129,Nazr Mohammed,1610612759,SAS,77,2004-05,1935,289,602,...,586,34,58,86,108,236,731,1000,0.32,0.54
144,1477,145,Bruce Bowen,1610612759,SAS,82,2004-05,2628,251,598,...,285,126,55,39,57,175,675,735,2.21,0.96
159,699,160,Brent Barry,1610612759,SAS,81,2004-05,1742,194,459,...,190,178,39,20,64,123,601,677,2.78,0.61
189,2484,189,Devin Brown,1610612759,SAS,67,2004-05,1237,173,409,...,176,92,39,12,53,83,494,497,1.74,0.74
196,2757,197,Beno Udrih,1610612759,SAS,80,2004-05,1148,173,390,...,83,150,41,10,77,96,471,439,1.95,0.53
207,109,208,Robert Horry,1610612759,SAS,75,2004-05,1396,157,375,...,268,80,67,60,69,138,451,616,1.16,0.97
227,1725,228,Rasho Nesterovic,1610612759,SAS,70,2004-05,1785,198,430,...,459,71,31,117,73,207,410,767,0.97,0.42


In [14]:
# Get the data for all nba teams
# Get all players for Coach Pops third championship as a head coach 2004-05
players_0607 = leagueleaders.LeagueLeaders(
    season='2006-07',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:600]
# Show the top 50 players
players_0607.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,977,1,Kobe Bryant,1610612747,LAL,77,3140,813,1757,0.463,...,439,413,111,36,255,205,2430,2129,1.62,0.43
1,2544,2,LeBron James,1610612739,CLE,78,3190,772,1621,0.476,...,526,470,125,55,250,171,2132,1997,1.88,0.50
2,2240,3,Gilbert Arenas,1610612764,WAS,74,2942,647,1548,0.418,...,338,443,139,13,236,249,2105,1789,1.88,0.59
3,1713,4,Vince Carter,1610612751,NJN,82,3126,726,1598,0.454,...,492,393,82,30,217,266,2070,1864,1.81,0.38
4,1717,5,Dirk Nowitzki,1610612742,DAL,78,2820,673,1341,0.502,...,693,263,52,62,167,171,1916,2098,1.57,0.31


In [15]:
# Get the spurs team for the 06-07 season
spurs_0607 = players_0607.loc[players_0607['TEAM']=='SAS']

spurs_0607.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
14,1495,15,Tim Duncan,1610612759,SAS,80,2726,618,1131,0.546,...,846,273,66,190,224,203,1599,2031,1.22,0.29
23,2225,24,Tony Parker,1610612759,SAS,77,2499,570,1096,0.520,...,250,420,82,6,191,137,1429,1394,2.20,0.43
47,1938,48,Manu Ginobili,1610612759,SAS,75,2060,396,854,0.464,...,327,263,109,27,157,155,1240,1299,1.68,0.69
135,714,136,Michael Finley,1610612759,SAS,82,1823,273,663,0.412,...,223,108,32,16,46,76,740,675,2.35,0.70
163,699,164,Brent Barry,1610612759,SAS,75,1631,202,425,0.475,...,160,138,56,12,61,88,635,703,2.26,0.92


In [16]:
# Add year column
spurs_0607.insert(6, 'Year', '2006-07')

spurs_0607

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,Year,MIN,FGM,FGA,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
14,1495,15,Tim Duncan,1610612759,SAS,80,2006-07,2726,618,1131,...,846,273,66,190,224,203,1599,2031,1.22,0.29
23,2225,24,Tony Parker,1610612759,SAS,77,2006-07,2499,570,1096,...,250,420,82,6,191,137,1429,1394,2.20,0.43
47,1938,48,Manu Ginobili,1610612759,SAS,75,2006-07,2060,396,854,...,327,263,109,27,157,155,1240,1299,1.68,0.69
135,714,136,Michael Finley,1610612759,SAS,82,2006-07,1823,273,663,...,223,108,32,16,46,76,740,675,2.35,0.70
163,699,164,Brent Barry,1610612759,SAS,75,2006-07,1631,202,425,...,160,138,56,12,61,88,635,703,2.26,0.92
194,1477,195,Bruce Bowen,1610612759,SAS,82,2006-07,2464,189,467,...,223,117,62,25,65,175,510,564,1.80,0.95
248,1922,249,Francisco Elson,1610612759,SAS,70,2006-07,1332,144,282,...,336,55,31,59,86,184,350,589,0.64,0.36
249,101177,250,Fabricio Oberto,1610612759,SAS,79,2006-07,1365,158,281,...,368,68,25,24,69,213,349,624,0.99,0.36
252,2757,253,Beno Udrih,1610612759,SAS,73,2006-07,948,127,344,...,82,122,27,1,56,70,340,292,2.18,0.48
277,2588,278,Matt Bonner,1610612759,SAS,56,2006-07,653,106,237,...,156,22,17,11,25,67,275,314,0.88,0.68


In [17]:
# Get the data for all nba teams
# Get all players for Coach Pops third championship as a head coach 2013-14
players_1314 = leagueleaders.LeagueLeaders(
    season='2013-14',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:600]
# Show the top 50 players
players_1314.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,201142,1,Kevin Durant,1610612760,OKC,81,3122,849,1688,0.503,...,598,445,103,59,285,174,2593,2572,1.56,0.36
1,2546,2,Carmelo Anthony,1610612752,NYK,77,2982,743,1643,0.452,...,622,242,95,51,198,224,2112,1942,1.22,0.48
2,2544,3,LeBron James,1610612748,MIA,77,2902,767,1353,0.567,...,533,488,121,26,270,126,2089,2255,1.81,0.45
3,201567,4,Kevin Love,1610612750,MIN,77,2797,650,1421,0.457,...,963,341,59,35,196,136,2010,2328,1.74,0.30
4,201933,5,Blake Griffin,1610612746,LAC,80,2863,718,1359,0.528,...,757,309,92,51,224,265,1930,2082,1.38,0.41


In [18]:
# Get the spurs team for the 06-07 season
spurs_1314 = players_1314.loc[players_1314['TEAM']=='SAS']

spurs_1314.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
51,2225,52,Tony Parker,1610612759,SAS,68,1997,456,914,0.499,...,155,388,36,9,151,86,1134,1067,2.57,0.24
53,1495,54,Tim Duncan,1610612759,SAS,74,2158,444,906,0.490,...,721,220,43,139,159,134,1119,1536,1.38,0.27
100,201158,99,Marco Belinelli,1610612759,SAS,80,2016,337,695,0.485,...,226,179,50,7,95,130,911,900,1.88,0.53
115,202695,116,Kawhi Leonard,1610612759,SAS,66,1923,337,645,0.522,...,412,133,114,50,80,127,844,1140,1.66,1.43
116,1938,117,Manu Ginobili,1610612759,SAS,68,1550,294,627,0.469,...,202,293,70,17,139,128,838,920,2.11,0.50


In [19]:
# Add year
spurs_1314.insert(6, 'Year', '2013-14')

spurs_1314

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,Year,MIN,FGM,FGA,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
51,2225,52,Tony Parker,1610612759,SAS,68,2013-14,1997,456,914,...,155,388,36,9,151,86,1134,1067,2.57,0.24
53,1495,54,Tim Duncan,1610612759,SAS,74,2013-14,2158,444,906,...,721,220,43,139,159,134,1119,1536,1.38,0.27
100,201158,99,Marco Belinelli,1610612759,SAS,80,2013-14,2016,337,695,...,226,179,50,7,95,130,911,900,1.88,0.53
115,202695,116,Kawhi Leonard,1610612759,SAS,66,2013-14,1923,337,645,...,412,133,114,50,80,127,844,1140,1.66,1.43
116,1938,117,Manu Ginobili,1610612759,SAS,68,2013-14,1550,294,627,...,202,293,70,17,139,128,838,920,2.11,0.50
118,201988,119,Patty Mills,1610612759,SAS,81,2013-14,1527,309,666,...,169,149,68,9,63,114,826,792,2.37,1.08
146,2564,147,Boris Diaw,1610612759,SAS,79,2013-14,1974,302,580,...,326,222,44,32,121,140,717,918,1.83,0.36
170,201980,171,Danny Green,1610612759,SAS,68,2013-14,1651,218,505,...,229,104,65,61,76,107,618,701,1.37,0.85
213,201168,214,Tiago Splitter,1610612759,SAS,59,2013-14,1271,181,346,...,363,90,29,31,75,117,483,704,1.20,0.39
258,202709,259,Cory Joseph,1610612759,SAS,68,2013-14,936,126,265,...,107,114,35,14,43,84,343,414,2.65,0.81


In [20]:
# Get the data for all nba teams
# Get all players for Coach Pops latest season
players_2223 = leagueleaders.LeagueLeaders(
    season='2022-23',
    season_type_all_star='Regular Season',
    stat_category_abbreviation='PTS'
).get_data_frames()[0][:600]
# Show the top 50 players
players_2223.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,1628369,1,Jayson Tatum,1610612738,BOS,74,2732,727,1559,0.466,...,649,342,78,51,213,160,2225,2209,1.61,0.37
1,203954,2,Joel Embiid,1610612755,PHI,66,2284,728,1328,0.548,...,670,274,66,112,226,205,2183,2369,1.21,0.29
2,1629029,3,Luka Doncic,1610612742,DAL,66,2391,719,1449,0.496,...,569,529,90,33,236,166,2138,2214,2.24,0.38
3,1628983,4,Shai Gilgeous-Alexander,1610612760,OKC,68,2416,704,1381,0.510,...,329,371,112,65,192,192,2135,2073,1.93,0.58
4,203507,5,Giannis Antetokounmpo,1610612749,MIL,63,2024,707,1278,0.553,...,742,359,52,51,246,197,1959,2072,1.46,0.21


In [21]:
# Get the spurs team for the 06-07 season
spurs_2223 = players_2223.loc[players_2223['TEAM']=='SAS']

spurs_2223.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
39,1629640,40,Keldon Johnson,1610612759,SAS,63,2063,503,1113,0.452,...,318,183,46,11,134,125,1385,1117,1.37,0.34
111,1630200,112,Tre Jones,1610612759,SAS,68,1984,341,743,0.459,...,245,448,89,9,110,98,875,1130,4.07,0.81
143,1628380,144,Zach Collins,1610612759,SAS,63,1441,284,548,0.518,...,402,180,37,49,129,199,731,972,1.40,0.29
152,1630170,153,Devin Vassell,1610612759,SAS,38,1178,261,595,0.439,...,148,136,43,17,57,57,703,634,2.39,0.75
166,1631103,167,Malaki Branham,1610612759,SAS,66,1550,269,611,0.440,...,181,127,33,7,76,115,673,591,1.67,0.43


In [22]:
# Add year
spurs_2223.insert(6, 'Year', '2022-23')

spurs_2223

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,Year,MIN,FGM,FGA,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
39,1629640,40,Keldon Johnson,1610612759,SAS,63,2022-23,2063,503,1113,...,318,183,46,11,134,125,1385,1117,1.37,0.34
111,1630200,112,Tre Jones,1610612759,SAS,68,2022-23,1984,341,743,...,245,448,89,9,110,98,875,1130,4.07,0.81
143,1628380,144,Zach Collins,1610612759,SAS,63,2022-23,1441,284,548,...,402,180,37,49,129,199,731,972,1.40,0.29
152,1630170,153,Devin Vassell,1610612759,SAS,38,2022-23,1178,261,595,...,148,136,43,17,57,57,703,634,2.39,0.75
166,1631103,167,Malaki Branham,1610612759,SAS,66,2022-23,1550,269,611,...,181,127,33,7,76,115,673,591,1.67,0.43
171,203926,172,Doug McDermott,1610612759,SAS,64,2022-23,1314,239,523,...,138,91,13,6,58,118,654,543,1.57,0.22
179,1628966,180,Keita Bates-Diop,1610612759,SAS,67,2022-23,1453,236,465,...,248,103,46,20,56,63,647,748,1.84,0.82
194,1631110,195,Jeremy Sochan,1610612759,SAS,56,2022-23,1458,250,552,...,295,142,43,23,97,138,614,683,1.46,0.44
215,1628984,216,Devonte' Graham,1610612759,SAS,73,2022-23,1338,161,431,...,122,196,45,16,53,79,539,562,3.70,0.85
296,1629641,297,Romeo Langford,1610612759,SAS,43,2022-23,844,121,259,...,114,50,24,13,30,67,297,313,1.67,0.80


In [23]:
# Join the tables
#  pd.merge(pd.merge(df1,df2,on='Courses'),df3,on='Courses')
spurs_data = [spurs_9697, spurs_9899, spurs_0203, spurs_0405, spurs_0607, spurs_1314, spurs_2223]


spurs_data = pd.concat(spurs_data)

spurs_data

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,Year,MIN,FGM,FGA,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
61,1122,62,Dominique Wilkins,1610612759,SAS,63,1996-97,1947,397,953,...,402,119,39,31,135,100,1145,976,0.88,0.29
91,137,92,Vernon Maxwell,1610612759,SAS,72,1996-97,2077,340,906,...,159,153,87,19,121,168,929,614,1.26,0.72
101,219,102,Vinny Del Negro,1610612759,SAS,72,1996-97,2238,365,781,...,210,231,59,7,92,131,886,868,2.51,0.64
118,422,119,Avery Johnson,1610612759,SAS,76,1996-97,2473,327,685,...,147,513,96,15,146,158,800,1004,3.51,0.66
151,61,151,Carl Herrera,1610612759,SAS,75,1996-97,1831,257,593,...,340,50,62,53,95,217,597,634,0.53,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,1631230,410,Dominick Barlow,1610612759,SAS,28,2022-23,408,46,86,...,102,24,10,19,15,56,110,203,1.60,0.67
457,203920,457,Khem Birch,1610612759,SAS,20,2022-23,162,19,32,...,25,7,5,5,8,24,43,63,0.88,0.63
472,1631160,472,Jordan Hall,1610612759,SAS,9,2022-23,83,9,28,...,12,11,1,0,7,6,28,24,1.57,0.14
473,1630563,472,Joshua Primo,1610612759,SAS,4,2022-23,93,9,26,...,13,18,1,2,11,8,28,32,1.64,0.09


In [24]:
# Save the data to use on tableau
spurs_data.to_csv('spurs_data.csv')

# Attempt of for loops for shot charts

In [25]:
# create a list to hold player ids for one year
player_id_9697 = spurs_9697.PLAYER_ID

# Show the list
player_id_9697

61     1122
91      137
101     219
118     422
151      61
152      42
154     251
155     724
160     781
234     246
307     994
326     764
337     293
391    1432
422    1118
Name: PLAYER_ID, dtype: int64

In [26]:
# Create empty list to store the dataframes
full_shot_chart = pd.DataFrame()

In [31]:
# Create the for loop
def full_shot_chart():
    
    for player in player_id_9697:
        shot_chart = shotchartdetail.ShotChartDetail(player_id = player,
                                                 team_id = 1610612759,
                                                 context_measure_simple= 'FGA',
                                                 season_type_all_star = 'Regular Season',
                                                 season_nullable = '1996-97').get_data_frames()[0]
    
    shot_charts = [shot_chart]
    
    full_shot_chart = pd.concat(shot_charts)
    
    full_shot_chart
    
    #full_shot_chart.append(shot_charts)

    print(full_shot_chart)

In [32]:
# Download to see how it looks in csv
full_shot_chart

<function __main__.full_shot_chart()>